# tDCS

## Load in data

This has to be converted to a fif first using the easy2fif.py script

In [9]:
import os, re, glob
import os.path as op
from MEEGbuddy import MEEGbuddy
from mne.io import Raw, RawArray
from mne import create_info, find_events
from MEEGbuddy.easy2fif import easy2fif
import numpy as np
from mne.io import Raw
import matplotlib.pyplot as plt
from pandas import read_csv, DataFrame

subject = '01'
session = '01'

####################

datarex = re.compile(r'[%s|%s]*.easy' % (subject.lower(), subject.upper()))
behrex = re.compile(r'[%s|%s]*.txt' % (subject.lower(), subject.upper()))
    
files = glob.glob(op.join('../data', subject, session, '*'))
if not files:
    print('Subject data must be in a folder with their name')


def sync_raw_behavior(raw, df):
    events = find_events(raw)
    events = events[events[:, 2] != 3]  # 3 is a marker for a break in the task
    first_stim = [i for i, e in enumerate(events[:, 2]) if e in  [11, 12, 21, 22]][0]
    # 11, 12, 21 and 22 are the stimuli, we don't want to start events until the first stimulus
    # before that there is some practice that gets recorded as triggers
    events = events[first_stim:]
    for i, e in enumerate(events[:, 2]):
        if e in [11, 12, 21, 22]:
            events[i, 2] = 50
        elif e in [1, 2]:
            events[i, 2] = 51
    selection = np.ones(len(events), dtype=bool)
    selection[1:] = events[1:, 2] != events[:-1, 2]  # delete all second, third + responses, we just want the first one
    selection[events[:, 2] == 50] = True
    events = events[selection]
    # due to a bug in the software several events are not recorded in the behavior that actually were recorded
    # in the eeg triggers. We have to go through and delete those
    counter = 0
    for i, t in enumerate(df.Type):
        if t == 'miss':
            if events[2*i + 1 - counter, 2] == 51:
                events = np.delete(events, 2*i + 1 - counter, axis=0)
            counter += 1 
    info = create_info(['Stimulus', 'Response'], raw.info['sfreq'], ['stim', 'stim'], verbose=False)
    arr = np.zeros((2, len(raw.times)))
    for i in events:
        arr[0 if i[2] == 50 else 1, i[0]:i[0]+100] = 1
    event_chs = RawArray(arr, info, verbose=False)
    raw = raw.drop_channels(['Events'])
    raw = raw.add_channels([event_chs])
    return raw, df


for f in files:
    if datarex.search(f):
        fdata = op.join(op.dirname(f), op.basename(f).split('.')[0] + '-raw.fif')
        behavior = op.join(op.dirname(f), op.basename(f).split('.')[0] + '.csv')
        if not op.isfile(fdata):
            raw = easy2fif(f)
            df = None
            for f in files:
                if behrex.search(f):
                    df = read_csv(f, delimiter='\t')
                    df = df.drop(0)
                    df = df.reset_index()
                    new_columns = {'Experiment': [], 'Flankers': [], 'Stimulus': [], 'Condition': []}
                    for code in df.Code:
                        experiment, flankers, stimulus, condition, _, _, _ = code.split(',')
                        new_columns['Experiment'].append(experiment)
                        new_columns['Flankers'].append(flankers)
                        new_columns['Stimulus'].append(stimulus)
                        new_columns['Condition'].append(condition)
                    df2 = DataFrame(new_columns)
                    df = df.join(df2, sort=False)
                    df['Trial'] = range(1, len(df) + 1)
                    df = df[['Trial', 'Type', 'Response', 'RT', 'Time', 'Duration',
                             'Experiment', 'Flankers', 'Stimulus', 'Condition']]
            if df is None:
                raise ValueError('No matching text file for behavior found')
            raw, df = sync_raw_behavior(raw, df)
            raw.save(fdata, overwrite=True)
            df.to_csv(behavior, index=False)
        df = read_csv(behavior)
        behavior_description = {'Trial': {'Baseline': {'Channel': 'Stimulus', 
                                                       'Time Min': -0.5, 
                                                       'Time Max': -0.1},
                                          'Events': {'Stimulus': {'Channel': 'Stimulus',
                                                                  'Time Min': -0.5,
                                                                  'Time Max': 1.0}},
                                          'Responses': {'Response': {'Channel': 'Response',
                                                                     'Time Min': -1.5,
                                                                     'Time Max': 0.5,
                                                                     'No Response': [i for i, r in enumerate(df.Response) 
                                                                                     if np.isnan(r)]}}},
                                'Type': 'Hit is when the subject is correct, ' + 
                                        'incorrect is when they press the wrong button ' +
                                        'and miss is when the subject does not respond',
                                'Response': '1.0 is for when the subject presses the left button ' + 
                                            '2.0 is for when the subject presses the right button',
                                'RT': 'Reaction time; how long it took the subject to respond ' +
                                      'after the stimulus was shown',
                                'Time': 'Overall time of the experiment in ms',
                                'Duration': 'How long the subject has to respond after the stimulus is shown',
                                'Experiment': 'Name of the experiment',
                                'Flankers': 'The two carrot characters on the left and right of ' + 
                                            'the target center character (all facing the same way)',
                                'Stimulus': 'The characters presented to the subject that trial',
                                'Condition': 'INC (incongruent) when the middle carrot is facing opposite the other carrots' +
                                             'CON (congruent) when the middle carrot and flankers are all facing the same direction'
                                }
        print('Loading in to MEEGbuddy %s %s' % (subject, session))
        data = MEEGbuddy(subject=subject, session=session, fdata=fdata, 
                         behavior=behavior, task='flanker', eeg=True, 
                         subjects_dir='../data', fs_subjects_dir=False, 
                         behavior_description=behavior_description)
        data.save2BIDS('../bids_data', overwrite=True)

Loading in to MEEGbuddy 01 01
Please provide the file for a boundary element model if you want source estimation, this can be done using a FLASH or T1 scan using MNE make_flash_bem or make_watershed_bem respectively
Please provide the file for a source space if you want source estimation, this can be done using MNE setup_source_space
Please provide the file for a coordinate transformation if you want source estimation, this can be done using MNE seting the SUBJECTS_DIR and SUBJECT environmental variables running 'mne_analyze', loading the subjects surface from the recon-all files and the digitization data from the raw file and then manually adjusting until the coordinate frames match. This can then be saved out as a coordinate transform file.


Once loaded once, this can be used instead

In [2]:
from MEEGbuddy import loadMEEGbuddy

subject = '01'
session = '01'

data = loadMEEGbuddy('../data', subject, session=session, task='flanker', eeg=True)

# Preprocess

## Mark Bad Channels for Proper Average Rereference
Select out bad channels by pressing the names on the left of the GUI or selecting the voltage trace

In [3]:
%matplotlib

data.plotRaw(keyword_out='Preprocessed', overwrite=True)

Using matplotlib backend: TkAgg
Loading Raw file(s)
../data/01/01/20190409094229_EEG_Flanker_pre_HD_HCa_01_01_1-raw.fif
File saved for Raw Preprocessed 


## Visualize Epochs

In [4]:
data.filterRaw(keyword_out='Preprocessed', l_freq=1, overwrite=True)
data.makeEpochs(keyword_in='Preprocessed', keyword_out='Vis', overwrite=True)
for event in data.getEvents():
    data.filterEpochs(event, keyword_in='Vis', h_freq=40, l_freq=None, overwrite=True) 
    data.plotEvoked(event, 'Condition', keyword='Vis', show=True, detrend=1, downsample=False, 
                    ylim={'eeg': [-50, 50]})

Loading Raw file(s)
../data/01/01/20190409094229_EEG_Flanker_pre_HD_HCa_01_01_1-raw.fif
File saved for Raw Preprocessed 
File loaded for Raw Preprocessed 
Baseline events found: 140
File saved for Epochs Baseline Vis 
Overwriting existing file.
File loaded for Epochs Baseline Vis 
Stimulus events found: 140
File saved for Epochs Stimulus Vis 
Overwriting existing file.
File loaded for Epochs Stimulus Vis 
File loaded for Epochs Stimulus Vis 
File saved for Epochs Stimulus Vis 
Overwriting existing file.
File loaded for Epochs Stimulus Vis 
File loaded for Epochs Baseline Vis 
File saved for Epochs Baseline Vis 
Overwriting existing file.
File loaded for Epochs Baseline Vis 


## ICA

In [5]:
data.findICA(keyword_in='Preprocessed', keyword_out='ICA',
             overwrite=True, overwrite_ica=True)

File loaded for Raw Preprocessed 
Using no channels as eogs
Using no channels as ecgs
eeg
File saved for ICA eeg ICA 
File saved for Raw ICA 


## Plot ICA for QC

In [ ]:
%matplotlib

data.plotICA(keyword_in='Preprocessed', keyword_out='ICA')

Using matplotlib backend: TkAgg
File loaded for Raw Preprocessed 
Using no channels as eogs
Using no channels as ecgs
File loaded for ICA eeg ICA 


## Make Epochs

In [ ]:
data.makeEpochs(keyword_in='ICA', overwrite=True)

## Check Epochs After ICA

In [ ]:
event = 'Stimulus'
data.filterEpochs(event, keyword_in='ICA', keyword_out='Vis',
                  h_freq=40, overwrite=True)
data.plotEvoked(event, keyword='Vis', ylim={'eeg': [-60, 40]}, 
                tmin=-0.5, tmax=0.5)

## Channel and Trial Rejection Using Autoreject

In [ ]:
for event in data.getEvents():
    data.markAutoReject(event, keyword_in='ICA', keyword_out='AR')

## Alternately Plot Epochs to Select Out Bads

In [ ]:
event = 'Stimulus'
data.plotEvoked(event, show=True, detrend=1, downsample=False, 
                ylim={'eeg': [-40, 40]})
data.plotEpochs(event, keyword_in='ICA', keyword_out='Clean', 
                l_freq=None, h_freq=None, overwrite=True)

## Final Lowpass/Bandstop Filter Epochs

In [ ]:
for event in data.getEvents():
    data.filterEpochs(event, h_freq=50, keyword_in='AR', keyword_out='AR_Filtered',
                      overwrite=True)

## Time Frequency Decomposition

In [ ]:
event = 'Stimulus'
condition = 'Condition'
data.makeWavelets(event, condition, keyword_in='AR',
                  keyword_out='TFR', power_type='pl',
                  overwrite=True)
data.makeWavelets(event,condition,keyword_in='AR',
                  keyword_out='TFR', power_type='npl', 
                  overwrite=True)

# Plots

## Control Variables

In [ ]:
data.plotControlVariables(conditions=['Type', 'Response',
                                      'Flankers', 'Stimulus', 'Condition'])

## Topo

In [ ]:
event = 'Stimulus'
condition = 'Condition'
data.plotTopo(event, condition=condition, keyword='AR_Filtered',
              show=True, detrend=1, downsample=False)

## TFR

In [ ]:
event = 'Stimulus'
condition = 'Condition'
data.plotTFR(event, condition, keyword='AR', tfr_keyword='TFR', power_type='pl')
data.plotTFR(event, condition, keyword='AR', tfr_keyword='TFR', power_type='npl')
data.plotTFR(event, condition, keyword='AR', tfr_keyword='TFR', bands=None, power_type='pl')
data.plotTFR(event, condition, keyword='AR', tfr_keyword='TFR', bands=None, power_type='npl')

# Analyses

## Cluster Permutation Test

In [ ]:
event = 'Stimulus'
condition = 'Condition'
data.CPT(event, condition, keyword_in='AR_Filtered',
         keyword_out='CPT', overwrite=True)
data.plotCPT(event, condition, keyword_in='AR_Filtered',
             cpt_p=0.001, keyword_out='CPT')

# Time Frequency Cluster Permutation

In [ ]:
event = 'Cue'
condition = 'Stimulus Type'

for value in data._default_values(condition): # pass one at at time to save on memory
    data.CPT(event, condition, values=[value], keyword_in='AR',
             keyword_out='TFR_CPT', power_type='npl', tfr=True,
             tfr_keyword='Autoreject', overwrite=True)
    data.CPTByBand(event, condition, values=[value], keyword_in='AR',
                   keyword_out='TFR_CPT', power_type='npl',
                   tfr_keyword='Autoreject', overwrite=True)

data.plotCPT(event, condition, keyword_in='AR', tfr=True,
             power_type='npl', keyword_out='CPT')
data.plotCPTByBand(event, condition, keyword_in='AR', tfr=True,
                   power_type='npl', keyword_out='CPT')

# Wavelet Connectivity

In [ ]:
event = 'Stimulus'
condition = 'Condition'
data.waveletConnectivity(event, condition, keyword_in='AR',
                         keyword_out='WC', 
                         bands={'theta': (4, 8), 'alpha': (8, 15),
                                'beta': (15, 35)}, downsample=False)